# **MiDeSec veri setindeki medikal/histopatolojik görüntüleri YOLO ile eğitime uygun hâle getirmek ve görüntü kalitesini standartlaştırmak.**

In [ ]:
!pip -q install ultralytics torchstain opencv-python

from google.colab import drive
drive.mount("/content/drive")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.0 MB/s eta 0:00:00
Mounted at /content/drive



 * pip install torchstain: Histopatoloji gibi görüntülerde boyama farklarını normalize etmek için (Macenko, Reinhard vb.).
 * pip install ultralytics: YOLOv8 (tespit/segmentasyon) kütüphanesi.



In [ ]:
import cv2
import torch
import torchstain
import shutil
import random
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from pathlib import Path


#### Kök yapısındaki MiDeSeC veri setimiz ve onun alt klasörleri train_images , test_images klasörlerimizin kontrollerini sağlayıp sorted ile yakalanan dosyaları alfabetik olarak sıralarız bu sayede eğitim yaparken verilerin okunması sırasında belirli düzen oluşturarak etiketler ile düzgün eşleşmesi sağlanır.
####
Train : 35 .jpg

Test : 15 .jpg

In [ ]:
DATA_DIR  = Path("/content/drive/MyDrive/Colab Notebooks/MiDeSeC")
train_src = DATA_DIR / "train_images"
test_src  = DATA_DIR / "test_images"

print("train_src:", train_src, train_src.exists()) # dosya kontrolü
print("test_src :", test_src,  test_src.exists())

train_jpg = sorted(train_src.glob("*.jpg"))
test_jpg  = sorted(test_src.glob("*.jpg"))
print("Train jpg:", len(train_jpg))
print("Test  jpg:", len(test_jpg))

assert len(train_jpg) > 0 and len(test_jpg) > 0, "train/test jpg bulunamadı"


train_src: /content/drive/MyDrive/Colab Notebooks/MiDeSeC/train_images True
test_src : /content/drive/MyDrive/Colab Notebooks/MiDeSeC/test_images True
Train jpg: 35
Test  jpg: 15


In [ ]:
# =========================
# Tüm cache temizliği
# =========================
for c in DATA_DIR.rglob("*.cache"):
    try:
        c.unlink()
        print("deleted cache:", c)
    except:
        pass
print("✅ Cache temizliği tamam.")


deleted cache: /content/drive/MyDrive/Colab Notebooks/MiDeSeC/labels/train.cache
deleted cache: /content/drive/MyDrive/Colab Notebooks/MiDeSeC/labels/val.cache
deleted cache: /content/drive/MyDrive/Colab Notebooks/MiDeSeC/labels/test.cache
✅ Cache temizliği tamam.


####Macenko yöntemi
Histopatolojik görüntülerde görülen boya (stain) kaynaklı renk varyasyonlarını gidermek amacıyla geliştirilmiş, istatistiksel ve fizik-tabanlı bir boya normalizasyon (stain normalization) tekniğidir. Yöntem, özellikle Hematoksilin–Eozin (H&E) boyalı doku kesitlerinde, farklı laboratuvar koşulları, tarayıcılar ve boyama protokollerinden kaynaklanan renk farklılıklarını standart bir renk uzayına eşlemek için kullanılır.

Temel mantık şudur: Tüm resimlerin renk tonunu (leke dağılımını), seçtiğin tek bir "referans resme"("PO_22.jpg") benzetmek.


Öncelikle eğitim verisi içerisinden seçilen bir referans görüntü kullanılarak normalizer fit edilmekte; ardından eğitim ve test klasörlerindeki tüm görüntüler, referansın boya uzayı ve yoğunluk istatistiklerine göre normalize edilerek ayrı hedef dizinlere (train_-images_macenko , test_images_macenko) kaydedilmektedir. İş akışı boyunca dönüşümler gerçekleştirmiş olup mantığı ise;

* OpenCV BGR okur ama biz o görüntüyü OpenCV’de “göstermek” için değil, başka bir kütüphaneye (torchstain / PyTorch pipeline) doğru renk sırasıyla vermek için RGB’ye çeviriyoruz.

* CHW dönüşümü için;

  Derin öğrenme kütüphaneleri genellikle: (C,H,W)

  Klasik görüntü kütüphaneleri ise : (H,W,C)

Bu adım, hesaplama verimliliği ve framework uyumluluğu açısından zorunludur.(tensör dönüşümleri (RGB, CHW)).

* Cihaz üzerinde hesaplama (CPU/GPU), normalizasyon sonrası elde edilen sürekli (float) piksel değerleri, görüntü format standartlarına uygun olacak şekilde 0–255 aralığına kırpılarak 8-bit tam sayıya dönüştürülmüştür, kanal/şekil bütünlüğü kontrolleri ve yazma sonrası doğrulama adımları uygulanarak, üretilecek veri kümesinin format tutarlılığı ve veri bütünlüğü garanti altına alınmaktadır. Bu yaklaşım, özellikle derin öğrenme tabanlı modellerde domain shift etkisini azaltarak genellenebilirliği ve eğitim stabilitesini artırmayı hedefler.

* cv2.imread → BGR -Normalizer için → RGB’ye çevir -Normalized çıktı (RGB) → dosyaya yazmak için BGR’ye geri çevir -cv2.imwrite



In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# =========================
# Macenko Normalizasyonu
# =========================
train_tgt = DATA_DIR / "train_images_macenko"
test_tgt  = DATA_DIR / "test_images_macenko"

# Eski varsa sil
for p in [train_tgt, test_tgt]:
    if p.exists():
        shutil.rmtree(p)
    p.mkdir(parents=True, exist_ok=True)

# Reference seçimi (var olduğundan emin ol)
ref_img_path = train_src / "P0_22.jpg"
if not ref_img_path.exists():
    # yoksa ilk train jpg'yi reference yap
    ref_img_path = train_jpg[0]
print("Reference:", ref_img_path)

normalizer = torchstain.normalizers.MacenkoNormalizer(backend="torch")

ref = cv2.cvtColor(cv2.imread(str(ref_img_path)), cv2.COLOR_BGR2RGB)
ref = torch.from_numpy(ref).permute(2, 0, 1).float().to(device)  # CHW, 0-255
normalizer.fit(ref)

def macenko_folder(src_dir: Path, dst_dir: Path):
    dst_dir.mkdir(parents=True, exist_ok=True)

    for p in tqdm(sorted(src_dir.glob("*.jpg")), desc=f"Macenko {src_dir.name}"):
        img = cv2.imread(str(p))
        assert img is not None, f"Okunamadı: {p}"
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        t = torch.from_numpy(img).permute(2, 0, 1).float().to(device)

        with torch.no_grad():
            out, _, _ = normalizer.normalize(t)

        # shape-safe
        if out.ndim == 3 and out.shape[0] == 3:
            out = out.permute(1, 2, 0)
        elif out.ndim == 2:
            out = out.unsqueeze(-1).repeat(1, 1, 3)

        out = out.clamp(0, 255).byte().cpu().numpy()
        assert out.ndim == 3 and out.shape[2] == 3, f"BOZUK SHAPE: {out.shape}"

        out = cv2.cvtColor(out, cv2.COLOR_RGB2BGR)
        out_path = dst_dir / p.name
        cv2.imwrite(str(out_path), out)

        chk = cv2.imread(str(out_path))
        assert chk is not None and chk.ndim == 3 and chk.shape[2] == 3 and chk.shape[1] > 10, \
            f"BOZUK DOSYA: {out_path} -> {chk.shape if chk is not None else None}"

macenko_folder(train_src, train_tgt)
macenko_folder(test_src,  test_tgt)

print("✅ Macenko images hazır.")


Reference: /content/drive/MyDrive/Colab Notebooks/MiDeSeC/train_images/P0_22.jpg


Macenko test_images: 100%|██████████| 15/15 [00:05<00:00,  2.60it/s]

✅ Macenko images hazır.


Bu kod bloğu, projenizde temiz bir sayfa açmak için kullanılan bir temizlik (cleanup) rutinidir. Özellikle YOLO gibi nesne tespiti modelleri eğitilirken, eski eğitimden kalan verilerin veya "cache" (önbellek) dosyalarının yeni eğitimi bozmaması için yapılır.
* Not: os.rmdir sadece boş klasörleri silebilirken, shutil.rmtree (remove tree) dolu klasörleri kökten silmek için kullanılır.
* val.cache: YOLO ve benzeri algoritmalar, etiketleri her seferinde baştan okumamak için bir "önbellek" dosyası oluşturur. Eğer veri setinizi güncellediyseniz ve bu dosya silinmezse, model eski verileri okumaya devam eder.
* .unlink(): pathlib kütüphanesinde bir dosyayı silmek için kullanılır. rmtree klasörler içindir, unlink ise tek bir dosyayı silmek için kullanılan güvenli bir yöntemdir.

In [ ]:
# =========================
# YOLO klasör yapısını sıfırla
# =========================
for p in [DATA_DIR/"images", DATA_DIR/"labels"]:
    if p.exists():
        shutil.rmtree(p)

# tekrar cache temizliği (güvenlik)
for c in DATA_DIR.rglob("*.cache"):
    try:
        c.unlink()
    except:
        pass

print("✅ images/ ve labels/ temizlendi.")


✅ images/ ve labels/ temizlendi.


###CSV Tabanlı Poligon Etiketlerinin YOLO Segmentasyon Formatına Dönüştürülmesi

Bu kod parçasında, görüntülere ait nesne sınırlarının CSV dosyaları içerisinde poligon (çokgen) koordinatları olarak tutulduğu bir veri yapısı, YOLO segmentasyon modelinin beklediği etiket formatına dönüştürülmüştür. YOLO mimarisi, giriş görüntüsünün çözünürlüğünden bağımsız çalışabilmek amacıyla, nesne koordinatlarını mutlak piksel değerleri yerine normalize edilmiş (0–1 aralığında) oransal değerler olarak kullanır. Bu nedenle, CSV dosyalarında piksel uzayında tanımlanmış olan koordinatlar, ilgili görüntünün genişlik (
𝑤
w) ve yükseklik (
ℎ
h) bilgileri kullanılarak normalize edilmiştir.

Dönüştürülen etiketler, YOLO segmentasyon formatına uygun olacak şekilde her satırda tek bir poligon olacak biçimde yazılmıştır. Kullanılan format aşağıdaki gibidir:

```
<class_id> <x1> <y1> <x2> <y2> ... <xN> <yN>
```
Bu veri setinde tek bir nesne sınıfı bulunduğundan, tüm poligonlar için sınıf kimliği sabit olarak 0 atanmıştır.

---

####Etiket Dizin Yapısının Oluşturulması

Kodun başlangıcında, eğitim, doğrulama ve test verileri için YOLO standartlarına uygun etiket dizinleri (labels/train, labels/val, labels/test) oluşturulmuştur. Bu adım, üretilen etiketlerin daha sonra doğrudan YOLO eğitim sürecinde kullanılabilmesini sağlamak amacıyla gerçekleştirilmiştir.

---

####CSV Satırlarının Poligonlara Dönüştürülmesi

CSV dosyalarındaki her satır, bir poligona ait koordinatları temsil edecek şekilde işlenmiştir. Okunan satırlarda bulunabilecek eksik veya geçersiz değerler (NaN) temizlenmiş, ardından koordinatların çiftler halinde
(
𝑥
,
𝑦
)
(x,y) olacak biçimde düzenlenmesi sağlanmıştır.

Bir poligonun geometrik olarak tanımlanabilmesi için en az üç noktaya ihtiyaç duyulması nedeniyle, altıdan az sayısal değere sahip satırlar geçersiz kabul edilerek işlem dışı bırakılmıştır. Ayrıca, koordinat sayısının tek olduğu durumlarda son değer kırpılarak veri bütünlüğü korunmuştur. Bu işlemler sonucunda her poligon,
[
𝑁
,
2
]
[N,2] boyutunda (N nokta, x–y koordinatları) bir yapı haline getirilmiştir.

---

####Poligon Koordinatlarının Normalize Edilmesi

YOLO segmentasyon formatına uyum sağlamak amacıyla her poligonun koordinatları normalize edilmiştir. Bu aşamada, poligon koordinatlarının zaten normalize edilmiş olup olmadığı kontrol edilmiştir. Eğer bir poligonun hem
𝑥
x hem de
𝑦
y koordinatlarının maksimum değeri 1’den küçük veya eşit ise, koordinatların zaten normalize olduğu kabul edilmiştir. Aksi durumda, koordinatlar piksel uzayında değerlendirilmiş ve aşağıdaki dönüşüm uygulanmıştır:

𝑥
′
=x/w

𝑦
′
=y/h


Bu yaklaşım, hem normalize edilmiş hem de piksel tabanlı CSV dosyalarının aynı kod altyapısı ile güvenli şekilde işlenebilmesini sağlamaktadır. Normalizasyon sonrası, olası etiketleme hatalarına karşı tüm koordinatlar
[
0
,
1
]
[0,1] aralığına kırpılmıştır. Bu adım, negatif değerler veya görüntü sınırlarını aşan koordinatların YOLO eğitim sürecini olumsuz etkilemesini önlemektedir.

---

####YOLO Segmentasyon Etiket Dosyalarının Yazılması

Normalize edilen poligonlar, YOLO segmentasyon formatına uygun olarak .txt dosyalarına yazılmıştır. Her poligon için önce sınıf kimliği, ardından poligon noktaları sırasıyla tek bir satır halinde dosyaya eklenmiştir. Koordinatlar, sayısal hassasiyet ile dosya boyutu arasında denge sağlamak amacıyla virgülden sonra altı basamak olacak şekilde yazılmıştır.

Geometrik olarak anlamlı olmayan, üç noktadan az poligonlar etiket dosyasına dahil edilmemiştir. Bu yaklaşım, model eğitiminde hatalı veya geçersiz segmentasyon maskelerinin kullanılmasını engellemektedir.

---

####Dönüşüm Sürecinin Orkestrasyonu

Tüm dönüşüm süreci, görüntü ve CSV dosyalarını eşleştiren bir üst seviye fonksiyon aracılığıyla yürütülmüştür. Bu fonksiyon, her görüntü için aynı ada sahip bir CSV dosyası aramakta; CSV bulunamayan veya geçerli poligon içermeyen örnekleri otomatik olarak atlamaktadır. Görüntülerin gerçek genişlik ve yükseklik bilgileri doğrudan dosya üzerinden okunarak normalizasyon işleminin doğru biçimde yapılması sağlanmıştır.

Başarılı şekilde dönüştürülen örnekler için etiket dosyaları oluşturulmuş, işlem sonunda kaç görüntü için etiket üretildiği ve kaç örneğin atlandığı raporlanmıştır. Bu sayede veri dönüşüm sürecinin kapsayıcılığı ve tutarlılığı nicel olarak izlenebilir hale getirilmiştir.


####Genel Değerlendirme

Bu etiket dönüşüm hattı, CSV tabanlı poligon açıklamalarını YOLO segmentasyon mimarisiyle tam uyumlu hale getirerek, farklı çözünürlüklere sahip görüntüler üzerinde tutarlı bir eğitim süreci yürütülmesini mümkün kılmaktadır. Normalize edilmiş koordinat kullanımı, modelin ölçekten bağımsız öğrenme yapabilmesini sağlarken; veri temizleme, sınırlandırma ve doğrulama adımları, eğitim sürecinde oluşabilecek hataların önüne geçmektedir.

In [ ]:
# =========================
# CSV -> YOLO Seg Label Dönüşümü
# =========================
labels_train = DATA_DIR / "labels" / "train"
labels_val   = DATA_DIR / "labels" / "val"
labels_test  = DATA_DIR / "labels" / "test"
labels_train.mkdir(parents=True, exist_ok=True)
labels_val.mkdir(parents=True, exist_ok=True)
labels_test.mkdir(parents=True, exist_ok=True)

def csv_rows_to_polygons(csv_path: Path):
    df = pd.read_csv(csv_path)
    polys = []
    for i in range(len(df)):
        row = df.iloc[i].to_numpy(dtype="float32")
        row = row[~np.isnan(row)]
        if len(row) < 6:
            continue
        if len(row) % 2 == 1:
            row = row[:-1]
        pts = row.reshape(-1, 2)
        polys.append(pts)
    return polys

def normalize_poly(poly_xy, w, h):
    poly = poly_xy.copy().astype("float32")
    maxx, maxy = np.nanmax(poly[:,0]), np.nanmax(poly[:,1])

    # daha sağlam karar:
    # Eğer hem x hem y <= 1 ise zaten normalize
    if not (maxx <= 1.0 and maxy <= 1.0):
        poly[:, 0] /= float(w)
        poly[:, 1] /= float(h)

    poly[:, 0] = np.clip(poly[:, 0], 0.0, 1.0)# taşmayı önler
    poly[:, 1] = np.clip(poly[:, 1], 0.0, 1.0)
    return poly

def write_yolo_seg_label(out_txt: Path, polygons, w, h, class_id=0):
    lines = []
    for poly in polygons:
        poly_n = normalize_poly(poly, w, h)
        if poly_n.shape[0] < 3:
            continue
        flat = poly_n.reshape(-1)
        lines.append(" ".join([str(class_id)] + [f"{v:.6f}" for v in flat.tolist()]))
    out_txt.write_text("\n".join(lines), encoding="utf-8")

def convert_with_csv(img_dir: Path, csv_dir: Path, out_dir: Path):
    ok, skip = 0, 0
    for img_path in sorted(img_dir.glob("*.jpg")):
        csv_path = csv_dir / f"{img_path.stem}.csv"
        if not csv_path.exists():
            skip += 1
            continue

        with Image.open(img_path) as im:
            w, h = im.size

        polys = csv_rows_to_polygons(csv_path)
        if len(polys) == 0:
            skip += 1
            continue

        out_txt = out_dir / f"{img_path.stem}.txt"
        write_yolo_seg_label(out_txt, polys, w, h, class_id=0)
        ok += 1

    print(img_dir.name, "-> labels yazılan:", ok, "skip:", skip)

# CSV'ler train_src/test_src içindeyse:
train_csv_dir = train_src
test_csv_dir  = test_src

convert_with_csv(train_tgt, train_csv_dir, labels_train)
convert_with_csv(test_tgt,  test_csv_dir,  labels_test)

print("labels/train:", len(list(labels_train.glob("*.txt"))))
print("labels/test :", len(list(labels_test.glob("*.txt"))))


train_images_macenko -> labels yazılan: 35 skip: 0
test_images_macenko -> labels yazılan: 15 skip: 0
labels/train: 35
labels/test : 15


Bu kod bloğunda
* images_train / images_test: Normalize edilmiş resimlerin (JPG) nihai olarak tutulacağı klasör yollarıdır.
glob ile kaynak klasöründeki tüm .jpg uzantılı dosyaları tek tek bulur ve shutil ile dosyayı kopyalar. Buradaki copy2'nin farkı, dosyanın oluşturulma ve değiştirilme tarihi gibi meta verilerini de beraberinde taşımasıdır.
* Hedef klasöre, dosyanın orijinal adıyla (p.name) kaydedilmesini sağlar.
* train_tgt: Bu Macenko işlemi görmüş resimlerinizin olduğu geçici klasördü.
Bu satırlarla birlikte resimler, etiketlerin (labels) yanındaki ilgili images klasörüne kopyalanmış olur.

Önemi! : YOLO eğitiminde her resmin (.jpg) tam olarak aynı isimde bir etiketi (.txt) olmalıdır. Eğer resim sayısı 100, etiket sayısı 90 ise, 10 tane resmin etiketi eksik demektir ve bu eğitimde hataya veya düşük performansa yol açabilir.

In [ ]:
# =========================
# images/ klasörlerine kopyala
# =========================
images_train = DATA_DIR / "images" / "train"
images_val   = DATA_DIR / "images" / "val"
images_test  = DATA_DIR / "images" / "test"
images_train.mkdir(parents=True, exist_ok=True)
images_val.mkdir(parents=True, exist_ok=True)
images_test.mkdir(parents=True, exist_ok=True)

def copy_jpg(src_dir: Path, dst_dir: Path):
    cnt = 0
    for p in src_dir.glob("*.jpg"):
        shutil.copy2(p, dst_dir / p.name)
        cnt += 1
    print(src_dir.name, "->", dst_dir, ":", cnt)

copy_jpg(train_tgt, images_train)
copy_jpg(test_tgt,  images_test)

print("images/train jpg:", len(list(images_train.glob("*.jpg"))))
print("labels/train txt:", len(list(labels_train.glob("*.txt"))))
print("images/test  jpg:", len(list(images_test.glob("*.jpg"))))
print("labels/test  txt:", len(list(labels_test.glob("*.txt"))))


train_images_macenko -> /content/drive/MyDrive/Colab Notebooks/MiDeSeC/images/train : 35
test_images_macenko -> /content/drive/MyDrive/Colab Notebooks/MiDeSeC/images/test : 15
images/train jpg: 35
labels/train txt: 35
images/test  jpg: 15
labels/test  txt: 15


In [ ]:
# =========================
# Train -> Val split
# =========================
all_train_imgs = sorted(list(images_train.glob("*.jpg")))
random.seed(42)
random.shuffle(all_train_imgs)

val_ratio = 0.30
val_count = max(1, int(len(all_train_imgs) * val_ratio))
val_files = all_train_imgs[:val_count]

for img_path in val_files:
    shutil.move(str(img_path), str(images_val / img_path.name))

    label_path = labels_train / f"{img_path.stem}.txt"
    if label_path.exists():
        shutil.move(str(label_path), str(labels_val / label_path.name))

print(f"✅ Val'a ayrılan: {len(val_files)}")
print("Kalan train:", len(list(images_train.glob("*.jpg"))), "labels:", len(list(labels_train.glob("*.txt"))))
print("Val        :", len(list(images_val.glob("*.jpg"))),   "labels:", len(list(labels_val.glob("*.txt"))))


✅ Val'a ayrılan: 10
Kalan train: 25 labels: 25
Val        : 10 labels: 10


### YOLO Segmentasyon Etiketlerinin Görsel Doğrulanması İçin GT Polygon Overlay (Debug) Modülü


Bu modülün amacı, YOLO segmentasyon formatında üretilen ground-truth (GT) poligon etiketlerinin doğru şekilde oluşturulup oluşturulmadığını hızlıca doğrulamak için, etiketleri ilgili görüntüler üzerine poligon çizimi (overlay) olarak bindirmektir. Özellikle veri dönüşüm adımlarında (CSV → YOLO segmentasyon) oluşabilecek normalize/denormalize hataları, nokta sırası problemleri, eksik/bozuk etiket dosyaları gibi durumlar model eğitimine geçmeden önce tespit edilebilmektedir. Bu nedenle modül, eğitim dizininden rastgele seçilen sınırlı sayıda örnek üzerinde (burada 20 adet) etiketi görselleştirerek kalite kontrol (quality assurance) işlevi görür.

---
####Çıktı Dizin Yönetimi (Temiz Başlangıç)

Kodun ilk bölümünde debug_overlay adlı bir çıktı klasörü oluşturulmakta; varsa daha önce üretilmiş debug çıktıları silinerek süreç tekrarlanabilir (reproducible) hale getirilmektedir. Bu yaklaşım, önceki çalıştırmalardan kalan görsellerin yeni çıktılarla karışmasını engeller ve değerlendirmeyi daha güvenilir kılar.


---
* read_yolo_poly(txt_path: Path)

Bu fonksiyon, YOLO segmentasyon etiket dosyasını (.txt) okuyarak içerdiği poligonları ayrıştırır ve bellekte işlenebilir bir forma dönüştürür.

Etiket dosyası boş ise, poligon listesi boş döndürülür. Böylece boş etiketli örneklerde hata üretilmeden akış devam eder.

Dosyadaki her satır, YOLO segmentasyon formatına göre şu şekilde yorumlanır:
<class_id> <x1> <y1> <x2> <y2> ...

Satırda en az üç nokta bulunması gereklidir (3 nokta = 6 koordinat + class id). Bu nedenle len(parts) < 7 koşulu, geometrik olarak anlamlı olmayan satırları elemek amacıyla kullanılır.

parts[1:] kısmı float değerlere dönüştürülerek (-1, 2) biçiminde yeniden şekillendirilir. Böylece her poligon, normalize koordinatlarla
[
𝑁
,
2
]
[N,2] formunda temsil edilir.

Sonuç olarak fonksiyon, her bir poligonu (class_id, xy) ikilileri halinde saklayan bir liste üretir. Bu yapı, sonraki aşamada denormalizasyon ve çizim işlemlerini kolaylaştırır.


---
* draw_polys(img_path: Path, txt_path: Path)

Bu fonksiyon, bir görüntü ve onun YOLO poligon etiket dosyasını alarak poligonları görüntü üzerine çizer ve görselleştirilmiş çıktıyı döndürür.

Görüntü OpenCV ile okunur ve görüntü boyutları (h, w) elde edilir.

read_yolo_poly fonksiyonu çağrılarak etiket dosyasındaki poligonlar normalize koordinatlar (0–1) biçiminde alınır.

YOLO’nun koordinatları normalize ettiği varsayımıyla, çizim yapabilmek için bu değerler tekrar piksel uzayına çevrilir:

𝑥
𝑝
𝑖
𝑥
𝑒
𝑙=
𝑥
𝑛
𝑜
𝑟
𝑚
×
𝑤


𝑦
𝑝
𝑖
𝑥
𝑒
𝑙=
𝑦
𝑛
𝑜
𝑟
𝑚
×
ℎ

Elde edilen noktalar tamsayıya (int32) çevrilir; bu adım OpenCV’nin çizim fonksiyonlarının beklediği veri tipiyle uyumluluk sağlar.

cv2.polylines ile poligon kapalı (isClosed=True) olacak şekilde görüntüye çizilir. Bu sayede segmentasyon sınırları görsel olarak incelenebilir.

Bu fonksiyonun çıktısı, etiket doğrulaması için kullanılacak overlay edilmiş görüntüdür.


---
####Rastgele Örnekleme ve Overlay Çıktılarının Üretilmesi

Ana bölümde, eğitim görüntüleri listesinden rastgele sıralama yapılarak örneklem alınır. Daha sonra:

Her görüntü için aynı isimli etiket dosyası aranır.

Etiket dosyası yoksa örnek atlanır (bu, veri setindeki eksik eşleşmelerin akışı kırmasını engeller).

Etiket dosyası mevcutsa draw_polys ile overlay görüntü oluşturulur ve debug_overlay klasörüne kaydedilir.

İşlem, maksimum 20 örnek üretildikten sonra durdurulur.

Son olarak kaç adet debug görseli üretildiği raporlanır. Bu çıktı, dönüşüm sürecinin “en azından belirli sayıda örnekte” etiket üretebildiğine dair pratik bir kontrol sağlar.

---

####Genel Değerlendirme

Bu debug modülü, YOLO segmentasyon etiketlerinin görsel doğrulanmasını sağlayarak veri hazırlama hattında kritik bir kalite kontrol adımı sunar. Özellikle normalize koordinatların doğru ölçeklenip ölçeklenmediği, poligonların görüntü sınırlarına oturup oturmadığı, hatalı nokta dizilimleri veya bozuk etiket dosyaları gibi problemler model eğitimi başlamadan önce tespit edilebilir. Bu yaklaşım, eğitim sürecinde oluşabilecek performans düşüşlerinin önemli bir kaynağı olan “yanlış etiket” problemini erken aşamada azaltmayı hedefler.

In [ ]:
# =========================
# Debug: 20 örnekte GT polygon overlay
# =========================
debug_dir = DATA_DIR / "debug_overlay"
if debug_dir.exists():
    shutil.rmtree(debug_dir)
debug_dir.mkdir(parents=True, exist_ok=True)

def read_yolo_poly(txt_path: Path):
    polys = []
    text = txt_path.read_text().strip()
    if not text:
        return polys
    for line in text.splitlines():
        parts = line.split()
        if len(parts) < 7:
            continue
        cls = int(float(parts[0]))
        xy = np.array([float(x) for x in parts[1:]], dtype=np.float32).reshape(-1, 2)
        polys.append((cls, xy))
    return polys

def draw_polys(img_path: Path, txt_path: Path):
    img = cv2.imread(str(img_path))
    h, w = img.shape[:2]
    polys = read_yolo_poly(txt_path)
    for cls, xy in polys:
        pts = xy.copy()
        pts[:,0] = pts[:,0] * w
        pts[:,1] = pts[:,1] * h
        pts = pts.astype(np.int32)
        cv2.polylines(img, [pts], isClosed=True, color=(0,255,0), thickness=2)
    return img

imgs = sorted(list(images_train.glob("*.jpg")))
random.shuffle(imgs)

count = 0
for img_path in imgs:
    txt_path = labels_train / f"{img_path.stem}.txt"
    if not txt_path.exists():
        continue
    vis = draw_polys(img_path, txt_path)
    cv2.imwrite(str(debug_dir / f"{img_path.stem}.jpg"), vis)
    count += 1
    if count >= 20:
        break

print("✅ Debug overlay çıktı klasörü:", debug_dir, "adet:", count)


✅ Debug overlay çıktı klasörü: /content/drive/MyDrive/Colab Notebooks/MiDeSeC/debug_overlay adet: 20


Bu kod bloğu, YOLO (You Only Look Once) modellerinin eğitimi için gerekli olan yapılandırma dosyasını (config file) oluşturur. YOLO, hangi resimlerin nerede olduğunu ve nesne isimlerini bu .yaml dosyasından okur.
* write_text: Aşağıdaki metni bir dosya olarak kaydeder.
* f""" (f-string): Metin içindeki {DATA_DIR} gibi değişkenlerin otomatik olarak gerçek yollara dönüşmesini sağlar.
* .strip(): Metnin başındaki ve sonundaki gereksiz boşlukları temizler.

In [ ]:
# =========================
# 9) YAML
# =========================
yaml_path = DATA_DIR / "midesec_train_val_test.yaml"
yaml_path.write_text(f"""
path: {DATA_DIR}
train: images/train
val: images/val
test: images/test

names:
  0: object
""".strip(), encoding="utf-8")

print(yaml_path.read_text())


path: /content/drive/MyDrive/Colab Notebooks/MiDeSeC
train: images/train
val: images/val
test: images/test

names:
  0: object


### YOLOv8n-Seg ile Aşama-1 (Stage-1) Eğitim Yapılandırması ve Gerekçeleri

Bu çalışma kapsamında, MiDeSeC veri seti üzerinde nesne segmentasyonu gerçekleştirmek amacıyla Ultralytics kütüphanesinin YOLOv8 segmentasyon modeli kullanılmıştır. Eğitim süreci, ön-eğitimli ağırlıklarla başlatılan ve küçük veri setlerinde aşırı uyum (overfitting) riskini azaltacak şekilde düzenlenen bir hiperparametre yapılandırması ile yürütülmüştür. Aşağıda sunulan kod, iki aşamalı eğitim stratejisinin ilk evresi (Stage-1) olup, modelin düşük seviyeli özellik çıkarıcı katmanlarının belirli bir bölümünün dondurulması (freeze) ile “stabil adaptasyon” hedefler.

ÖNEMLİ! Klasik augment ham görüntü de dönüşümler yaparken mask/bbox gibi etiketleri yapmıyor ve bu durum etiket-görüntü uyuşmazlıklarına sebepiyat vererek modelin yanlış öğrenmesini sağlar. Bu kodda ise YOLO nun özelliklerinden olan ❤ :

Ultralytics YOLO’nun mosaic/mixup/flip/translate/scale/degrees augment’leri label-aware çalışır: mask/polygon/bbox etiketlerini görüntüyle birlikte aynı dönüşüme uğratır ayrıca histopatolojiye özel olarak stain normalizasyonu ve kontrollü stain/renk varyasyonu tercih ediyorum.

#### Epoch, Görüntü Boyutu ve Batch
* epochs=60: Küçük veri setlerinde, modelin veri dağılımına yeterince uyum sağlayabilmesi için makul bir eğitim süresi sunar. Çok düşük epoch sayıları underfitting’e, çok yüksek epoch sayıları ise overfitting’e yol açabilir; bu değer, erken durdurma (patience) ile birlikte dengeli bir tercihtir.

* imgsz=640: YOLO ailesinde yaygın kullanılan bir giriş çözünürlüğüdür. Segmentasyonda nesne sınır detayları önemli olduğundan, çok düşük çözünürlük mask kalitesini düşürebilir; 640 genellikle ayrıntı ile hız arasında uygun bir denge sağlar.

* batch=8: GPU belleği kısıtları ve segmentasyon maskelerinin daha fazla bellek tüketmesi nedeniyle batch küçük tutulmuştur. Küçük batch boyutları bazı durumlarda daha gürültülü gradyanlar üretebilir; burada AdamW ve düzenli augment ile stabilite desteklenmiştir.


---


#### Veri Artırma (Augmentation): Küçük Veri İçin “Sakin” Ayarlar
Küçük veri setlerinde veri artırma, genellenebilirliği artırır; ancak agresif augment, özellikle tıbbi/histopatolojik görüntülerde anlamsal bozulma yaratabilir. Bu nedenle parametreler “düşük şiddetli” seçilmiştir.

* mosaic=0.05: Mosaic, dört görüntüyü birleştirerek bağlam çeşitliliği sağlar; fakat segmentasyonda maske geometrisini karmaşıklaştırabilir. Bu nedenle oran çok düşük tutulmuştur.

* mixup=0.0 & copy_paste=0.0: Bu teknikler segmentasyon maskelerinde yapay birleşimler üretebilir ve küçük veri setinde etiket gürültüsü riskini artırabilir. Bu nedenle devre dışı bırakılmıştır.

* fliplr=0.5: Yatay çevirme, veri seti simetrisi uygunsa güçlü ve güvenli bir augmenttir. Olasılık 0.5 seçilerek çeşitlilik sağlanır.

* translate=0.03: Görüntüyü %3 oranında kaydırma; konum değişimlerine karşı dayanıklılık kazandırır ancak aşırı kayma nesneleri çerçeve dışına itebileceği için düşük tutulmuştur.

* scale=0.15: %15 ölçek değişimi, boyut varyasyonlarını öğretir; segmentasyonda maske konturlarını çok bozmayacak ılımlı bir değerdir.

* degrees=2.0: Küçük dönüşler (±2°) gerçekçi varyasyon sunar; büyük dönüşler ise özellikle mikroskobik/medikal görüntülerde gerçek dışı geometriye yol açabilir.

* hsv_h=0.01, hsv_s=0.15, hsv_v=0.12: Renk/aydınlatma varyasyonlarına karşı dayanıklılık için sınırlı renk jitter uygulanır. Histopatolojide renk duyarlılığı yüksek olduğundan hue çok düşük, satürasyon ve parlaklık daha kontrollü seçilmiştir.

* auto_augment=None & erasing=0.0: Otomatik augment ve random erasing, segmentasyon sınırlarında yapay artefakt üretebilir; bu nedenle kullanılmamıştır.


---



#### Optimizasyon ve Düzenlileştirme (Regularization)
* AdamW: Adam tabanlı yöntem, küçük veri setlerinde hızlı yakınsama sağlar. AdamW’nin “decoupled weight decay” yaklaşımı, ağırlık çürümesini daha doğru uygulayarak genellenebilirliği destekler.

* lr0=0.002: Başlangıç öğrenme oranı; çok yüksek lr eğitim kararsızlığına, çok düşük lr yavaş öğrenmeye yol açar. Bu değer, küçük batch ve freeze stratejisi ile uyumludur.

* cos_lr=True: Cosine öğrenme oranı zamanlaması, eğitim ilerledikçe lr’ı yumuşak biçimde azaltarak daha stabil bir optimizasyon sunar.

* weight_decay=0.0015: Aşırı uyumu azaltmaya dönük düzenlileştirme terimidir. Özellikle küçük veri setlerinde false positive artışını kontrol altına almak için kullanışlıdır.
freeze ile modelin ilk 10 katmanını (çoğunlukla backbone’un erken blokları) dondurarak eğitim sırasında güncellenmesini engeller.
patience ile modelin doğrulama performansı belirli bir süre iyileşmezse eğitim erken sonlandırılır.

####Genel Sonuç

Bu yapılandırma, ön-eğitimli YOLOv8n-seg modelini MiDeSeC veri setine uyarlamak için; kontrollü veri artırma, AdamW + weight decay ile düzenlileştirme, cosine lr zamanlaması ve backbone dondurma (freeze) gibi stratejileri bir arada kullanmaktadır. Özellikle küçük veri senaryolarında bu yaklaşım, hem eğitimin stabil ilerlemesini hem de modelin genellenebilirliğini artırmayı hedefler.

In [ ]:
from ultralytics import YOLO
from pathlib import Path
import torch

from ultralytics import YOLO
from pathlib import Path

model = YOLO("yolov8n-seg.pt")

results_stage1 = model.train(
    data=str(yaml_path),
    epochs=60,
    imgsz=640,
    batch=8,
    device=device,

    project="runs/segment",
    name="midesec_y8nseg",

    # küçük veri için daha sakin augment
    augment=True,
    mosaic=0.05,
    mixup=0.0,
    copy_paste=0.0,
    fliplr=0.5,
    translate=0.03,
    scale=0.15,
    degrees=2.0,
    hsv_h=0.01, hsv_s=0.15, hsv_v=0.12,

    auto_augment=None,
    erasing=0.0,

    # düzenlileştirme (false positive azaltır)
    optimizer="AdamW",
    lr0=0.002,
    cos_lr=True,
    weight_decay=0.0015,

    # 🔥 2 aşamalı eğitimin farkı burada
    freeze=10,
    patience=30,
)

best1 = Path(results_stage1.save_dir) / "weights" / "best.pt"
print("✅ Stage-1 best:", best1)



Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=None, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/MiDeSeC/midesec_train_val_test.yaml, degrees=2.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.01, hsv_s=0.15, hsv_v=0.12, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.002, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=0.05, multi_scale=False, name=midesec_y8nseg4, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=Tru

### YOLOv8 Segmentasyon Modelinde Aşama-2 (Stage-2) Fine-Tuning Yapılandırması ve Gerekçeleri

Bu aşama, iki aşamalı eğitim stratejisinin ikinci evresi olup, Stage-1’de elde edilen en iyi model ağırlıklarının üzerine inşa edilen ince ayar (fine-tuning) sürecini temsil eder. Stage-1’de modelin erken katmanları (backbone’un bir bölümü) dondurularak veri setine kontrollü adaptasyon sağlanmış; Stage-2’de ise bu kısıt kaldırılarak modelin tüm parametrelerinin hedef veri setine daha hassas biçimde uyumlanması amaçlanmıştır. Bu yaklaşım, küçük veri setlerinde hem stabil yakınsama hem de genelleme performansı açısından yaygın olarak tercih edilen bir “progressive unfreezing” stratejisidir.

#### Eğitim Süresi ve Girdi Ölçekleri
* epochs=40: Fine-tuning aşamasında amaç “yeniden öğrenmek” değil, öğrenilmiş temsilleri hedef alana göre rafine etmektir. Bu nedenle Stage-1’e kıyasla daha kısa bir epoch sayısı yeterli olur. Aynı zamanda uzun fine-tuning, küçük veri setinde aşırı uyumu artırabilir.

* imgsz=640: Stage-1 ile aynı çözünürlüğün korunması, öğrenilmiş özelliklerin ölçek tutarlılığını sağlar ve eğitim dinamiklerini daha öngörülebilir kılar.

* batch=8: Segmentasyon modelleri bellek maliyetli olduğundan ve Stage-1 ile tutarlılık istenildiğinden aynı batch boyutu korunmuştur. Bu, hiperparametre değişkenliğini azaltarak Stage-1/Stage-2 kıyasını daha adil hale getirir.


---



#### Deney Yönetimi (Run Organizasyonu)
Bu ayar, fine-tuning koşusunu ayrı bir “run” adıyla kaydederek:

Stage-1 ve Stage-2 çıktılarını ayrıştırır,

karşılaştırmalı değerlendirmeyi kolaylaştırır,

deneyselliği (experiment tracking) artırır.


---



#### Fine-Tuning İçin Kontrollü Augmentation Stratejisi
Stage-2’de augment ayarları Stage-1’e göre daha da “sakinleştirilmiştir”. Bunun temel nedeni şudur:

* Fine-tuning, modelin karar sınırlarını küçük ve anlamlı düzeltmelerle iyileştirmeyi hedefler; agresif augment bu aşamada gereksiz gürültü üretip optimumdan uzaklaştırabilir.

* mosaic=0.0: Mosaic, görüntüleri birleştirerek güçlü çeşitlilik sağlar; ancak segmentasyonda maske geometrisini yapaylaştırabilir. Fine-tuning aşamasında, modelin gerçek veri dağılımına daha yakın örnekler görmesi istenir; bu yüzden mosaic kapatılmıştır.

* mixup/copy_paste=0.0: Segmentasyon maskelerinde yapay birleşimler üretip etiket dağılımını bozabileceği için devre dışı bırakılmıştır.

* fliplr=0.5: Yatay çevirme, genellikle semantiği bozmayan güvenli bir artırmadır; çeşitliliği korumak için tutulmuştur.

* translate=0.02, scale=0.10, degrees=1.0: Stage-1’e göre daha düşük tutulmuştur. Fine-tuning’de amaç, nesnelerin sınırlarını hassaslaştırmak olduğu için büyük geometrik dönüşümler yerine küçük sapmalarla dayanıklılık korunur.

* HSV jitter (hsv_s/hsv_v): Stage-1’e kıyasla azaltılmıştır. Tıbbi/histopatoloji benzeri alanlarda renk bilgisi önemli olduğundan, aşırı renk perturbasyonu doku özelliklerini bozabilir. Hue düşük tutulurken satürasyon ve parlaklık daha kontrollü seçilmiştir.

* auto_augment=None, erasing=0.0: Otomatik ve rastgele silme tabanlı yöntemler, fine-tuning’de segmentasyon kontur hassasiyetini olumsuz etkileyebileceği için kapalıdır.


---



#### Optimizasyon: Daha Düşük Öğrenme Oranı ile Hassas Uyum
* AdamW: Küçük veri setlerinde stabil ve hızlı yakınsama sağlar; ayrıca weight decay’i daha doğru uyguladığı için genelleme açısından tercih edilir.

* lr0=0.001 (LR düşürme): Stage-2’nin en kritik farkı, öğrenme oranının düşürülmesidir. Fine-tuning’de yüksek LR:

öğrenilmiş iyi temsilleri “yıkabilir” (catastrophic forgetting),

doğrulama performansında dalgalanma yaratabilir.
Düşük LR ise optimum çevresinde küçük adımlarla daha kontrollü iyileştirme sağlar.

* cos_lr=True: Cosine zamanlama, LR’ı eğitim boyunca yumuşak biçimde azaltarak son epoch’larda daha ince ayar yapılmasına katkı sağlar.

* weight_decay=0.0015: Stage-1 ile tutarlı tutulmuştur; bu düzenlileştirme terimi, özellikle küçük veri setlerinde aşırı uyumu ve false positive eğilimini azaltmaya yardımcı olur.


---



#### Genel Sonuç

Stage-2 yapılandırması, Stage-1’de elde edilen optimal ağırlıklardan başlayarak, daha düşük öğrenme oranı ve daha az agresif veri artırma ile modelin karar sınırlarını rafine etmeyi hedefleyen bir fine-tuning sürecidir. Bu aşamada backbone’un serbest bırakılması (freeze=0), modelin tüm katmanlarının hedef veri setine uyumlanmasını mümkün kılarken; LR düşürme, weight decay ve early stopping birlikte kullanılarak küçük veri setine bağlı overfitting riski kontrol altında tutulmaktadır. Bu strateji, pratikte segmentasyon doğruluğunun ve özellikle sınır hassasiyetinin artırılmasına katkı sağlayan güvenilir bir eğitim yaklaşımıdır.

In [ ]:
model = YOLO(str(best1))  # Stage-1'in en iyi ağırlığından devam

results_stage2 = model.train(
    data=str(yaml_path),
    epochs=40,
    imgsz=640,
    batch=8,
    device=device,

    project="runs/segment",
    name="midesec_y8nseg_ft",  # fine-tune run adı

    augment=True,
    mosaic=0.0,          # fine-tune'da mosaic'i kapatmak genelde iyi
    mixup=0.0,
    copy_paste=0.0,
    fliplr=0.5,
    translate=0.02,
    scale=0.10,
    degrees=1.0,
    hsv_h=0.01, hsv_s=0.10, hsv_v=0.10,

    auto_augment=None,
    erasing=0.0,

    optimizer="AdamW",
    lr0=0.001,           # 🔥 LR düşür
    cos_lr=True,
    weight_decay=0.0015,

    freeze=0,            # 🔥 artık backbone serbest
    patience=20,
)

best2 = Path(results_stage2.save_dir) / "weights" / "best.pt"
print("✅ Stage-2 best:", best2)


Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=None, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/MiDeSeC/midesec_train_val_test.yaml, degrees=1.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.01, hsv_s=0.1, hsv_v=0.1, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/runs/segment/midesec_y8nseg4/weights/best.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=midesec_y8nseg_ft, nbs=64, nms=False, opset=None, optimize=Fal